In [1]:
# one off operation
#!pip install azure-identity
#!pip install duckdb

Register an App
https://fivetran.com/docs/destinations/onelake/setup-guide

In [2]:
import duckdb
from   azure.identity   import ClientSecretCredential
import configparser

In [3]:
#one off operation , hopefully will not be required in the next version of duckDB
#duckdb.sql(" install azure ;force install delta from core_nightly ; ")

In [4]:
config = configparser.ConfigParser()
config.read("C:/KV/variable.ini")
credential = ClientSecretCredential(
                client_id     =config.get("myvars", "appId"),
                client_secret =config.get("myvars", "secret") , 
                tenant_id     =config.get("myvars", "tenantId")
                )
token =       credential.get_token("https://storage.azure.com/.default").token

In [5]:
def attach_lakehouse(workspace,lakehouse,token):
  duckdb.sql(f"""          
    load azure ;
    CREATE or replace SECRET secret4 ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}');
    set enable_object_cache = true 
     """)
  tables = duckdb.sql(f""" SELECT distinct(split_part(file, '/', 7)) as tables FROM glob
                      ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/*/_delta_log/*.json")
                    """).df()['tables'].tolist()
  for tbl in tables:
    print(tbl)
    duckdb.sql(f""" FROM delta_scan("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/{tbl}")  """).to_view(tbl)
 

In [6]:
# use Workspace and lakehouse name, token either using notebookutils inside Fabric
#  or register an App for access outside Fabric
attach_lakehouse("aemodev","storage",token)

calendar
mstdatetime
price
scada
duid
states


In [7]:
duckdb.sql(""" 
           select Region, sum(INITIALMW)/12 as MWH from scada s
            join duid d
            on s.DUID = d.DUID
            where settlementdate > '2024-09-01'
            GROUP BY ALL
            order by MWH desc
           """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┬────────────────────┐
│ Region  │        MWH         │
│ varchar │       double       │
├─────────┼────────────────────┤
│ NSW1    │ 1778308.1934894912 │
│ QLD1    │   1692673.04212748 │
│ VIC1    │  1507637.076776429 │
│ SA1     │ 345265.95433918276 │
│ TAS1    │  303055.9516300151 │
│ WA1     │  41197.83914295663 │
└─────────┴────────────────────┘